In [1]:
%load_ext autoreload
%autoreload 2

import random
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Function
import optuna
from modules.utils import evaluate_model
import matplotlib.pyplot as plt

# dataset related
from modules import CompetitionDataset, load_combined_moabb_data
from torch.utils.data import DataLoader, TensorDataset
from moabb.datasets import BNCI2014_001, PhysionetMI, Cho2017, Weibo2014 # 250 hz

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

c:\Users\ahmad\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = '/content/drive/MyDrive/ai_data/eeg_detection/data/mtcaic3'
# model_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/models/ssvep.pth'
# optuna_db_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/optuna/optuna_studies.db'
data_path = './data/mtcaic3'
model_path = './checkpoints/mi/the_honored_one.pth'
optuna_db_path = './checkpoints/mi/optuna/the_honored_one.db'

In [3]:
batch_size = 64
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [4]:
moabb_train_datasets = [
    PhysionetMI(imagined=True),  # 109 subjects
    Weibo2014(),  # 10 subjects, 64 channels
    CompetitionDataset(),
]
train_val = [CompetitionDataset(split="validation")]

# Load combined data
X_train, class_labels_train, domain_labels_train, info_train = load_combined_moabb_data(
    datasets=train_datasets,
    paradigm_config={
        "channels": ["Cz", "C3", "C4"],
        "tmin": 0.0,
        "tmax": 4.0,
        "resample": 250,
    },
)


# Load combined data
X_val, y_val, domain_labels_val, info_val = load_combined_moabb_data(
    datasets=train_val,
    paradigm_config={
        "channels": ["Cz", "C3", "C4"],
        "tmin": 0.0,
        "tmax": 4.0,
        "resample": 250,
    },
)

X_train = torch.from_numpy(X_train).float()  # FloatTensor of shape (N, C, T)
y_train = torch.from_numpy(y_train).long()  # LongTensor of shape (N, 2)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

X_train_t = torch.from_numpy(X_train).float()       # FloatTensor of shape (N, C, T)
y_train_t = torch.from_numpy(y_train).long()        # LongTensor of shape (N, 2)
y_train_t[:, 1] -= 1 

train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

X_val_t = torch.from_numpy(X_val).float()
y_val_t = torch.from_numpy(y_val).long()

val_dataset = TensorDataset(X_val_t, y_val_t)
val_loader  = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



Processing dataset: BNCI2014_001
Original subject range: 1 to 9


/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~580 KiB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~580 KiB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~580 KiB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 24 events (all good)

Adjusted subject range: (1, 9)
Number of trials: 2592
Number of subjects: 9

Processing dataset: BNCI2014_004
Original subject range: 1 to 9


/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 120 events (all good), 3 – 7 s (baseline off), ~2.8 MiB, data loaded,
 'left_hand': 60
 'right_hand': 60>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 120 events (all good), 3 – 7 s (baseline off), ~2.8 MiB, data loaded,
 'left_hand': 60
 'right_hand': 60>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 160 events (all good), 3 – 7 s (baseline off), ~3.7 MiB, data loaded,
 'left_hand': 80
 'right_hand': 80>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 160 events (all g

Adjusted subject range: (10, 18)
Number of trials: 6520
Number of subjects: 9

Processing dataset: Zhou2016
Original subject range: 1 to 4
Reading 0 ... 305029  =      0.000 ...  1220.116 secs...


...

Adjusted subject range: (19, 22)
Number of trials: 1199
Number of subjects: 4

Processing dataset: CompetitionDataset
Original subject range: 1 to 20


No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 0 – 4 s (baseline off), ~242 KiB, data loaded,
 'left_hand': 6
 'right_hand': 4>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 0 – 4 

Adjusted subject range: (23, 52)
Number of trials: 2400
Number of subjects: 30

=== COMBINED DATASET SUMMARY ===
Total trials: 12711
Feature shape: (12711, 3, 1001)
Class distribution: [6343 6368]
Subject range: 1 to 52
Total unique subjects: 52

Processing dataset: CompetitionDataset
Original subject range: 1 to 30


/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 0 – 4 s (baseline off), ~242 KiB, data loaded,
 'left_hand': 6
 'right_hand': 4>
  warn(f"warnEpochs {epochs}")
No stim channel nor annotations found, skipping setting annotations.
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 0 – 4 s (baseline off), ~242 KiB, data loaded,
 'left_hand': 4
 'right_hand': 6>
  warn(f"warnEpochs {epochs}")
No stim channel nor annotations found, skipping setting annotations.
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 0 – 4 s (baseline off), ~242 KiB, data loaded,
 'left_hand': 6
 'right_hand': 4>
  warn(f"warnEpochs {epochs}")





WARNING TASK DF EMPTY 6 AT ROW 36 AT SPLIT validation






WARNING TASK DF EMPTY 7 AT ROW 37 AT SPLIT validation






WARNING TASK DF EMPTY 8 AT ROW 38 AT SPLIT validation






WARNING TASK DF EMPTY 9 AT ROW 39 AT SPLIT validation






WARNING TASK DF EMPTY 10 AT ROW 40 AT SPLIT validation






WARNING TASK DF EMPTY 11 AT ROW 41 AT SPLIT validation






WARNING TASK DF EMPTY 12 AT ROW 42 AT SPLIT validation






WARNING TASK DF EMPTY 13 AT ROW 43 AT SPLIT validation






WARNING TASK DF EMPTY 14 AT ROW 44 AT SPLIT validation






WARNING TASK DF EMPTY 15 AT ROW 45 AT SPLIT validation






WARNING TASK DF EMPTY 16 AT ROW 46 AT SPLIT validation






WARNING TASK DF EMPTY 17 AT ROW 47 AT SPLIT validation






WARNING TASK DF EMPTY 18 AT ROW 48 AT SPLIT validation






WARNING TASK DF EMPTY 19 AT ROW 49 AT SPLIT validation






WARNING TASK DF EMPTY 20 AT ROW 50 AT SPLIT validation






WARNING TASK DF EMPTY 21 AT ROW 51 AT SPLIT validation






WARNING TASK DF EMPTY 22 

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class DepthWiseConv2D(nn.Module):
    def __init__(self, in_channels, kernel_size, dim_mult=1, padding=0, bias=False):
        super(DepthWiseConv2D, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels * dim_mult, padding=padding, kernel_size=kernel_size, groups=in_channels, bias=bias)

    def forward(self, x: torch.Tensor):
        return self.depthwise(x)


class SeperableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, bias=False):
        super(SeperableConv2D, self).__init__()
        self.depthwise = DepthWiseConv2D(in_channels, kernel_size, padding=padding)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

class SSVEPClassifier(nn.Module):
    # EEG Net Based
    # todo look at this https://paperswithcode.com/paper/a-transformer-based-deep-neural-network-model
    def __init__(self, n_electrodes=16, out_dim=4, dropout=0.25, kernLength=256, F1=96, D=1, F2=96, hidden_dim=100, layer_dim=1):
        super().__init__()

        # B x C x T
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            #
            DepthWiseConv2D(F1, (n_electrodes, 1), dim_mult=D, bias=False),
            nn.BatchNorm2d(F1*D),
            nn.ELU(),
            nn.MaxPool2d((1, 2)), # todo try making this max pool
            nn.Dropout(dropout),
            #
            SeperableConv2D(F1 * D, F2, kernel_size=(1, 16), padding='same', bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout),
        )

        self.lstm_head = LSTMModel(F2, hidden_dim, layer_dim, out_dim)

    def forward(self, x: torch.Tensor):
        """expected input shape: BxCxT"""
        x = x.unsqueeze(1)
        y = self.block_1(x) # B x F1 x 1 x time_sub

        y = y.squeeze(2) # B x F1 x time_sub
        y = y.permute(0, 2, 1) # B x time_sub x F1
        y = self.lstm_head(y)

        return y

    def forward(self, x):
        # x: B×C×F×T  (from your CWT)
        seq = self.feature_extractor(x)  # B×T_sub×feat_dim
        class_out, last_time_step = self.label_lstm(seq)  # B×out_dim, Bxhidden_dim

        domain_out = self.domain_classifier(self.grl_layer(last_time_step))
        return class_out, domain_out
        
    def set_grl_alpha(self, new_grl):
        self.grl_layer = GradientReversal(alpha=new_grl)
        

n_electrodes = 3
dummy_x = torch.randn(5, 3, 10001).to(device)
model = DANN_SSVEPClassifier(
    dropout=0.26211635308091535,
    n_electrodes=n_electrodes,
    out_dim=2,
    domain_classes=52,
    kernLength=8,
    F1=8,
    D=2,
    hidden_dim=256,
    layer_dim=1,
    grl_alpha=0,
    domain_lstm_div=2,
).to(device)

model(dummy_x)

(tensor([[-0.0352, -0.0377],
         [ 0.0156, -0.0649],
         [ 0.0232, -0.0725],
         [ 0.0234, -0.0517],
         [ 0.0053, -0.0590]], grad_fn=<AddmmBackward0>),
 tensor([[-0.0807,  0.0608,  0.0397,  0.0816,  0.0437, -0.0154,  0.1028,  0.0384,
           0.0008, -0.0467, -0.0799,  0.0024, -0.0168,  0.0402, -0.0747, -0.0197,
           0.0145, -0.0753, -0.0800,  0.0624, -0.1098,  0.0332, -0.0610,  0.0238,
           0.0758,  0.0417, -0.0126,  0.0564, -0.0250,  0.0588,  0.1071,  0.0346,
           0.0015,  0.0806,  0.0073, -0.0373,  0.0586,  0.0810,  0.0502, -0.0515,
          -0.0196, -0.0869,  0.0356, -0.0707, -0.0795, -0.0103,  0.0392,  0.0290,
          -0.0150,  0.0874,  0.0650, -0.1045],
         [-0.0747,  0.0528,  0.0406,  0.0596,  0.0251, -0.0195,  0.0831,  0.0424,
          -0.0044, -0.0441, -0.0665,  0.0050, -0.0354,  0.0348, -0.0660, -0.0433,
           0.0187, -0.0511, -0.0674,  0.0598, -0.1265,  0.0102, -0.0914,  0.0216,
           0.0706,  0.0519, -0.0249,  0.06

In [7]:
avg_losses_label = []
avg_losses_domain = []
val_label_accuracies = []
train_label_accuracies = []
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [ ]:
try:
    model.load_state_dict(torch.load(model_path, weights_only=True))
except Exception:
    print("skipping model loading...")


opt = torch.optim.Adam(model.parameters(), lr=0.003746351873334935)
criterion = nn.CrossEntropyLoss()
epochs = 300
domain_loss_weight = 0.2

epochs = 4000
for epoch in range(epochs):
    avg_loss = 0
    correct = 0
    total = 0
    model.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device).to(torch.int64)
        y_pred = model(x).to(device)

        loss = criterion(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        avg_loss += loss.item()

        # Training accuracy calculation
        _, pred = torch.max(y_pred, 1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    avg_loss /= len(train_loader)
    avg_losses.append(avg_loss)
    avg_losses_label.append(avg_loss)
    train_acc = 100.0 * correct / total
    train_label_accuracies.append(train_acc)

    avg_loss_label /= len(train_loader)
    avg_loss_domain /= len(train_loader)
    avg_losses_label.append(avg_loss_label)
    avg_losses_domain.append(avg_loss_domain)
    train_label_acc = 100.0 * correct_label / total
    train_domain_acc = 100.0 * correct_domain / total
    train_label_accuracies.append(train_label_acc)
    train_domain_accuracies.append(train_domain_acc)

    if (epoch + 1) % 5 == 0 or True:
        label_evaluation, domain_evaluation = evaluate_model(model, val_loader, device)
        val_label_accuracies.append(label_evaluation)
        val_domain_accuracies.append(domain_evaluation)
        model.cpu()
        torch.save(model.state_dict(), model_path)
        model.to(device)
        print(
            f"Epoch {epoch+1:2d}/{epochs} | "
            f"Label Loss: {avg_loss_label:.4f} | "
            f"Domain Loss: {avg_loss_domain:.4f} | "
            f"Train Label Acc: {train_label_acc:.2f}% | "
            f"Train Domain Acc: {train_domain_acc:.2f}% | "
            f"Val Label Acc: {label_evaluation*100:.2f}% | "
            f"Val Domain Acc: {domain_evaluation*100:.2f}% | "
            f"LR: {opt.param_groups[0]['lr']:.6f} | "
            f"GRL: {new_grl_alpha:.6f}"
        )


skipping model loading...


In [1]:
epochs = range(1, len(avg_losses_label) + 1)
val_epochs = range(0, len(val_label_accuracies) * 5, 5)  # if you log every 5 epochs

plt.figure(figsize=(16, 10))

# 1. Label Loss
plt.subplot(2, 2, 1)
plt.plot(epochs, avg_losses_label, label='Train Label Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Label Loss')
plt.legend()

# 2. Domain Loss
plt.subplot(2, 2, 2)
plt.plot(epochs, avg_losses_domain, label='Train Domain Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Domain Loss')
plt.legend()

# 3. Label Accuracy
plt.subplot(2, 2, 3)
plt.plot(epochs, train_label_accuracies, label='Train Label Acc')
plt.plot(val_epochs, [v*100 for v in val_label_accuracies], label='Val Label Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Label Accuracy')
plt.legend()

# 4. Domain Accuracy
plt.subplot(2, 2, 4)
plt.plot(epochs, train_domain_accuracies, label='Train Domain Acc')
plt.plot(val_epochs, [v*100 for v in val_domain_accuracies], label='Val Domain Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Domain Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'avg_losses_label' is not defined

In [ ]:
class CustomTrainer(Trainer):
    # Called by _objective during an Optuna trial
    def prepare_trial_run(self):
        assert isinstance(self.trial, optuna.Trial), "Trial not set!"

        # 1) Hyperparameter search space
        # Data params
        window_length = self.trial.suggest_categorical("window_length", [128, 256, 640])
        batch_size    = self.trial.suggest_categorical("batch_size", [32, 64])

        # Model extractor params (based on EEG3D+MDD)
        kernLength = self.trial.suggest_categorical("kernLength", [8, 16, 32, 64, 128])
        F1         = self.trial.suggest_categorical("F1", [8, 16, 32, 64])
        D          = self.trial.suggest_categorical("D", [1, 2, 4])
        F2         = self.trial.suggest_categorical("F2", [16, 32, 64, 128])
        dropout    = self.trial.suggest_float("dropout", 0.1, 0.5)
        # MDD head params
        hidden_dim = self.trial.suggest_categorical("hidden_dim", [64, 128, 256])
        layer_dim  = self.trial.suggest_int("layer_dim", 1, 3)
        # MDD alignment weight
        lambda_mdd = self.trial.suggest_float("lambda_mdd", 0.1, 1.0)

        # Optimizer
        lr = self.trial.suggest_float("lr", 1e-5, 1e-3, log=True)

        # 2) Prepare data
        super()._prepare_data(is_trial=True,
                              batch_size=batch_size,
                              window_length=window_length)

        # 3) Build MDD model
        extractor_kwargs = dict(
            n_electrodes=self.data.num_channels,
            kernLength=kernLength,
            F1=F1,
            D=D,
            F2=F2,
            dropout=dropout,
        )
        lstm_kwargs = dict(
            input_dim=F2 * (self.data.freq_bins // D),  # adjust if freq_bins variable
            hidden_dim=hidden_dim,
            layer_dim=layer_dim,
            output_dim=self.data.num_classes,
        )
        self.model = MDD_SSVEPClassifier(
            extractor_kwargs=extractor_kwargs,
            lstm_kwargs=lstm_kwargs
        ).to(self.device)
        self.lambda_mdd = lambda_mdd

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    # Called by train() for final run
    def prepare_final_run(self):
        study      = self._get_study()
        best       = study.best_params
        # Data
        super()._prepare_data(is_trial=False)
        # Build final model
        extractor_kwargs = dict(
            n_electrodes=self.data.num_channels,
            kernLength=best["kernLength"],
            F1=best["F1"],
            D=best["D"],
            F2=best["F2"],
            dropout=best["dropout"],
        )
        lstm_kwargs = dict(
            input_dim=best["F2"] * (self.data.freq_bins // best["D"]),
            hidden_dim=best["hidden_dim"],
            layer_dim=best["layer_dim"],
            output_dim=self.data.num_classes,
        )
        self.model = MDD_SSVEPClassifier(
            extractor_kwargs=extractor_kwargs,
            lstm_kwargs=lstm_kwargs
        ).to(self.device)
        # load weights?
        try:
            self.model.load_state_dict(torch.load(self.model_path))
            print(f"Loaded weights from {self.model_path}")
        except:
            print("No checkpoint found, training from scratch.")

        lr = best["lr"]
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='max', factor=0.5, patience=20,
            threshold=1e-4, threshold_mode='rel',
            cooldown=0, min_lr=1e-6
        )

trainer = CustomTrainer(
        data_path=data_path,
        optuna_db_path=optuna_db_path,
        model_path=model_path,
        train_epochs=500, # Final training epochs
        tune_epochs=50,   # Epochs per trial
        optuna_n_trials=50,
        task="mi",
        eeg_channels=eeg_channels,
        data_fraction=0.4
    )

In [ ]:
delete_existing = False
trainer.optimize(delete_existing)

In [ ]:
trainer.train()

In [ ]:
trainer._prepare_training(False)
trainer.model.eval()
f"test accuracy: {evaluate_model(trainer.model, trainer.eval_loader, device)}"